# Libraries

In [ ]:
from  google.colab  import  drive 
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import scipy
import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,balanced_accuracy_score
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics.pairwise as pw
from sklearn.metrics.pairwise import pairwise_distances
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns
from collections import Counter
import random
from scipy import sparse
from scipy.stats import pearsonr
import re

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import operator

import nltk
from nltk.corpus import stopwords
import torch
from torchtext import data
from torchtext import datasets

In [ ]:
! pip install transformers -q
from transformers import pipeline

     |████████████████████████████████| 5.5 MB 4.2 MB/s 
     |████████████████████████████████| 163 kB 47.3 MB/s 
     |████████████████████████████████| 7.6 MB 44.8 MB/s 


In [ ]:
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

# Recommendations from Keywords

## With Emotions

In [ ]:
stop = set(stopwords.words("english")) #setting stopwords

In [ ]:
def cosine_similarity_of(text1, text2):
        
        # taking words
        first = re.compile(r"[\w']+").findall(text1)
        second = re.compile(r"[\w']+").findall(text2)

        # creating dictionary from each word and count
        vector1 = Counter(first)
        vector2 = Counter(second)

        # changing vectors to sets to find comman words with intersection
        common = set(vector1.keys()).intersection(set(vector2.keys()))

        dot_product = 0.0

        for i in common:
            # getting amount of each common word for vectors and multiply them also add them
            dot_product += vector1[i] * vector2[i]

        squared_sum_vector1 = 0.0
        squared_sum_vector2 = 0.0

        # getting summation of word counts for every vector
        for i in vector1.keys():
            squared_sum_vector1 += vector1[i]**2

        for i in vector2.keys():
            squared_sum_vector2 += vector2[i]**2

        # calculating magnitude of vector with squared sums
        magnitude = math.sqrt(squared_sum_vector1) * math.sqrt(squared_sum_vector2)

        if not magnitude:
            return 0.0
        else:
            return float(dot_product) / magnitude

In [ ]:
# Για αυτό το racommend δεν χρησιμοποιείται, αλλά έχει αντικατασταθεί από την get_rating_weight_with_threshold
def rating_weight(rating,Q):
    '''calculating rating weight'''
    w = (2*Q/5)*rating - Q
    return w

In [ ]:
def calculate_final_score(cos_sim, w):
# ανάθεση βάρους στο cosine similarity
        effect = (cos_sim / 100) * w
        return cos_sim + effect

In [ ]:
def get_rating_weight_with_threshold(rating, count, threshold, Q):
            # rating effect 
            w = (2*Q/5)*rating - Q
            # multiplier effect
            M = math.exp((-threshold*0.68)/count)

            return w * M

In [ ]:
#αποθηκεύεται για να χρησιμοποιηθεί για το user interface
df = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Restaurants Reviews/restaurants_for_keywords_recommendation.csv') 

In [ ]:
df

,Name,UserId,Review,Rating,Emotion,sentiment,Review_Clean,Reviewer_Count,Restaurant_Count,positive,word_without_stop
0,aleria,1,a full 5 stars! phenomenal michelin ranked res...,5,admiration,2,a full 5 stars phenomenal michelin ranked rest...,30,886,True,full 5 stars phenomenal michelin ranked restau...
1,lithos,285,"amazing food, amazing service! a place that wi...",5,admiration,2,amazing food amazing service a place that will...,30,1005,True,amazing food amazing service place complete va...
2,lithos,257,great restaurant with delicious food and great...,5,admiration,2,great restaurant with delicious food and great...,30,1005,True,great restaurant delicious food great service ...
3,lithos,258,"great food and great service minded staff, hig...",5,admiration,2,great food and great service minded staff high...,30,1005,True,great food great service minded staff highly r...
4,lithos,259,we visited this restaurant in july 2021. we we...,5,admiration,2,we visited this restaurant in july 2021 we wer...,30,1005,True,visited restaurant july 2021 family 5 tables s...
...,...,...,...,...,...,...,...,...,...,...,...
26203,gods_restaurant,1005,"xcellent food, had a chicken souvlaki and grea...",5,admiration,2,xcellent food had a chicken souvlaki and great...,11,1005,True,xcellent food chicken souvlaki great service p...
26204,karamanlidika,1005,this place is spectacular. recommended by two...,5,admiration,2,this place is spectacular recommended by two ...,11,1005,True,place spectacular recommended two athenian fri...
26205,efcharis,1005,we visited efxaris for lunch. great lamb shop...,4,admiration,2,we visited efxaris for lunch great lamb shops...,11,1005,True,visited efxaris lunch great lamb shops great c...
26206,oroscopo,1005,my 11 year old son picked this restaurant befo...,5,admiration,2,my 11 year old son picked this restaurant befo...,11,1005,True,11 year old son picked restaurant came athens ...


In [ ]:
def get_recommendations_with_emotion(df,keywords):

        score_dict = {}

        emotion_from_keywords = emotion(keywords)[0]['label']
        print('Emotion is ' + emotion_from_keywords)

        for index, row in df.iterrows():
            cs = cosine_similarity_of(row['word_without_stop'], keywords)

            rating = row['Rating']
            rating_count = row['Restaurant_Count']
            emotion_from_comment = row['Emotion']


            if row['positive']==True:
                pos_rat = (7+rating)/2
            else:
                pos_rat = rating/2
                
            threshold=110
            # I choose Q as 10 to not to change weight too much
            rating_contribution = get_rating_weight_with_threshold(pos_rat,rating_count,threshold,10)

            score_from_similarity = calculate_final_score(cs, rating_contribution)

            # Αν το συναίσθημα από τα keywords ισοδυναμεί με το συναίσθημα του σχολίου τότε το score αυξάνεται κατά 10%
            if(emotion_from_comment == emotion_from_keywords):
              last_score = score_from_similarity + 0.2*score_from_similarity
            else:
              last_score = score_from_similarity

            score_dict[index] = last_score
            
        #sort τα σχόλια
        sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)

        counter = 0

        #δημιουργία νέου dataframe για την αποθήκευση των recommendations
        result = pd.DataFrame(columns=('Name', 'score'))

        # τα reviews με τα υψηλότερα cosign similarity
        for i in sorted_scores:
            result = result.append({'Name': df.iloc[i[0]]['Name'],
                                        'score': i[1]}, ignore_index=True)
            
            counter += 1

            if counter>20:
                break

        result = result.drop_duplicates(subset=['Name'], keep='first')
        result = result.reset_index(drop=True)   
        return result

In [ ]:
keywords = "happy waiters"
recommendations = get_recommendations_with_emotion(df,keywords)
recommendations[:15]

Emotion is joy


,Name,score
0,elaea,0.606347
1,maiandros,0.532668
2,oroscopo,0.504080
3,arcadia,0.471524
4,lithos,0.451983
5,ydria,0.412599
6,gods_restaurant,0.399501
7,liondi,0.391429
8,nolan,0.333189
9,tzitzikas_kai_mermigas,0.327641
